In [8]:
# Conceptual Setup: Agent as a Tool
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.tools import agent_tool
from pydantic import BaseModel

In [9]:
from google import adk
from google.adk.agents import Agent
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part
from google.adk.sessions import VertexAiSessionService
from google.adk.tools import google_search

In [10]:
# Define a target agent (could be LlmAgent or custom BaseAgent)
class ImageGeneratorAgent(BaseAgent): # Example custom agent
    name: str = "ImageGen"
    description: str = "Generates an image based on a prompt."
    # ... internal logic ...
    async def _run_async_impl(self, ctx): # Simplified run logic
        prompt = ctx.session.state.get("image_prompt", "default prompt")
        # ... generate image bytes ...
        image_bytes = b"..."
        yield Event(author=self.name, content=types.Content(parts=[types.Part.from_bytes(image_bytes, "image/png")]))

image_agent = ImageGeneratorAgent()
image_tool = agent_tool.AgentTool(agent=image_agent) # Wrap the agent

# Parent agent uses the AgentTool
artist_agent = LlmAgent(
    name="Artist",
    model="gemini-2.0-flash-preview-image-generation",
    instruction="Create a prompt and use the ImageGen tool to generate the image.",
    tools=[image_tool] # Include the AgentTool
)
# Artist LLM generates a prompt, then calls:
# FunctionCall(name='ImageGen', args={'image_prompt': 'a cat wearing a hat'})
# Framework calls image_tool.run_async(...), which runs ImageGeneratorAgent.
# The resulting image Part is returned to the Artist agent as the tool result.

In [11]:
app = AdkApp(
   agent=artist_agent,
    enable_tracing=True# Required.
   # session_service_builder=session_service_builder,  # Optional.
)

In [12]:
session = app.create_session(user_id='user')

In [13]:
session

Session(id='ba1fb305-78f4-4fa0-b2c3-761eef6bebc0', app_name='default-app-name', user_id='user', state={}, events=[], last_update_time=1753532418.8021355)

In [14]:
for event in app.stream_query(
    user_id="user",
    # session_id=SESSION_ID, # Optional. you can pass in the session_id when querying the agent
    message="Generate the any laptop image",
):
    print(event)

Exception in thread Thread-15 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/google/adk/runners.py", line 158, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/opt/conda/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 649, in run_until_complete
    return future.result()
  File "/opt/conda/lib/python3.10/site-packages/google/adk/runners.py", line 146, in _invoke_run_async
    async for event in self.run_async(
  File "/opt/conda/lib/python3.10/site-packages/google/adk/r